In [1]:
import os
import glob
import json
import pickle
import numpy as np
import pandas as pd
import datasetdatabase as dsdb

print(dsdb.version.VERSION)

os.environ["DSDB_PROCESS_LIMIT"] = "16"

1.0.31.stable


Reading the config file

In [7]:
with open("../qcb-drug-config.json", "r") as fjson:
    config_json = json.load(fjson)
print(json.dumps(config_json, indent=2))

{
  "meta": "QCB_drug_cell_meta",
  "id": "cell_id",
  "cell_info": "/allen/aics/assay-dev/Analysis/QCB_DRUG_database/cell_info",
  "raw_prefix": "raw.ome.tif",
  "seg_prefix": "seg.ome.tif",
  "mem_channel": 1,
  "dna_channel": 0,
  "str_channel": 2,
  "run": [
    {
      "status": "off",
      "structure_name": "cell",
      "extra_features": [],
      "description": "Cell features extracted from QCB DRUG images",
      "save_as": "QCB_DRUG_CELL_feature.pkl"
    },
    {
      "status": "off",
      "structure_name": "dna",
      "extra_features": [],
      "description": "Nucleus features extracted from QCB DRUG images",
      "save_as": "QCB_DRUG_DNA_feature.pkl"
    },
    {
      "status": "on",
      "structure_name": "tubulin",
      "extra_features": [
        "network"
      ],
      "description": "Features extracted from QCB DRUG images of TUBA1B-tagged microtubules",
      "save_as": "QCB_DRUG_TUBA1B_feature.pkl"
    },
    {
      "status": "off",
      "structure_name":

Loading metadata

In [8]:
with open(os.path.join("../../data-raw/",config_json["meta"]+".pkl"), "rb") as fp:
    df_meta = pickle.load(fp)
df_meta.head()

,cell_id,cell_ver,czi_filename,drug_label,idx_in_stack,roi,str_ver,structure_name
0,0ade9e16-db16-420a-9ccb-8cad3b32ee04,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,1,"[35, 141, 113, 380, 264, 469]",0.1.0,tubulin
1,ca1f5d66-da95-4208-bd89-65c6c614a9f1,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,2,"[35, 139, 183, 427, 327, 608]",0.1.0,tubulin
2,662de948-69c0-4b93-92f8-f3bcc7e38655,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,3,"[35, 139, 225, 458, 23, 406]",0.1.0,tubulin
3,7806500f-6df9-4b9f-8b23-fde6fd09d40e,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,4,"[35, 144, 403, 529, 156, 439]",0.1.0,tubulin
4,17fcb999-72a5-425d-b485-b02edb16651c,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,9,"[35, 147, 293, 450, 168, 397]",0.1.0,tubulin


In [9]:
df_meta.shape

(1519, 8)

Connecting to database

In [10]:
prod = dsdb.DatasetDatabase(config=config_json["database"])

Uploading datasets

In [11]:
for struct in config_json["run"]:
    
    if struct["status"] == "on":
        
        ds_name = struct["save_as"].replace(".pkl","")
        
        print("running:", struct["structure_name"], "save as", ds_name)

        try:
            prod._purge_dataset(name=ds_name)
            print("\t dataset",ds_name,"purged")
        except: pass
        
        with open(os.path.join("../../data-raw/",struct["save_as"]), "rb") as fp:
            df = pickle.load(fp)
  
        df = df.reset_index(drop=False)
        df = df.rename(index=str, columns={"index":"cell_id"})
        ds = dsdb.Dataset(df, name=ds_name, description=struct["description"])

        ds.upload_to(prod)

running: tubulin save as QCB_DRUG_TUBA1B_feature
	 dataset QCB_DRUG_TUBA1B_feature purged
Storing algorithm parameters.
Input dataset already exists in database. 1
Tearing down object...
[============================================================] 100.0% 228/228 ~ 0:0:0 remaining
